<a href="https://colab.research.google.com/github/RaoBurhan001/Deep_Learning/blob/main/Diarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Resemblyzer
!pip install pydub
!pip install spectralcluster
!pip install ffmpeg-python


     |████████████████████████████████| 15.7 MB 170 kB/s 
     |████████████████████████████████| 78 kB 7.1 MB/s 
     |████████████████████████████████| 66 kB 4.6 MB/s 
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp37-cp37m-linux_x86_64.whl size=72494 sha256=970a7e5e10406337e1b5b2fd699b57a96f91077dfdfe4ded9e117f33eb4ee6fc
  Stored in directory: /root/.cache/pip/wheels/11/f9/67/a3158d131f57e1c0a7d8d966a707d4a2fb27567a4fe47723ad
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26324 sha256=32169eba541e8811591c885688dc9d32b49be590aa6e7af1c26b5f8e8faa4d17
  Stored in directory: /root/.cache/pip/wheels/35/f3/15/01aa6571f0a72ee6ae7b827c1491c37a1f72d686fd22b43b0e
Successfully built webrtcvad typing


In [ ]:
""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path
from spectralcluster import SpectralClusterer

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))
  audio_file_path = audio
  wav_fpath = audio

  wav = preprocess_wav(wav_fpath)
  encoder = VoiceEncoder("cpu")
  _, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)
  print(cont_embeds.shape)
  clusterer = SpectralClusterer(
    min_clusters=2,
    max_clusters=100,
    
   
    
    )

  labels = clusterer.predict(cont_embeds)
  labelling = create_labelling(labels,wav_splits)
  print(labelling)
  return audio, sr


In [ ]:
audio, sr = get_audio()


Loaded the voice encoder model on cpu in 0.01 seconds.
(361, 256)
[('1', 0, 3.74), ('0', 3.74, 4.58), ('1', 4.58, 5.6), ('0', 5.6, 9.86), ('1', 9.86, 9.92), ('0', 9.92, 9.98), ('1', 9.98, 10.28), ('0', 10.28, 10.46), ('1', 10.46, 10.7), ('0', 10.7, 14.84), ('1', 14.84, 14.9), ('0', 14.9, 15.08), ('1', 15.08, 15.14), ('0', 15.14, 15.5), ('1', 15.5, 15.68), ('0', 15.68, 18.86), ('1', 18.86, 20.18), ('0', 20.18, 22.4)]


In [ ]:
import numpy
print(audio)
a= numpy.array(audio)
print(a)
for i in a:
  print(i)

Streaming output truncated to the last 5000 lines.
21
24
26
35
39
43
43
34
30
28
23
17
12
7
8
10
11
15
19
25
35
39
41
42
43
44
38
34
32
31
31
31
35
41
50
54
56
56
52
57
53
38
25
13
16
16
7
3
5
22
35
39
37
38
45
36
16
-10
-32
-35
-39
-46
-55
-55
-40
-26
-14
-12
-14
-4
3
-5
-22
-34
-40
-38
-36
-38
-37
-24
-6
5
7
13
15
11
4
-4
-13
-19
-23
-26
-19
-6
-1
6
15
23
34
40
32
22
21
16
8
5
2
0
4
7
6
5
10
20
19
12
16
19
18
16
18
20
17
20
18
15
18
18
17
14
16
21
22
25
27
32
34
32
33
27
27
31
32
34
33
40
45
41
39
36
34
31
24
16
13
12
7
8
9
14
24
25
28
34
40
48
49
48
49
55
63
66
68
69
67
60
53
44
25
10
1
-5
-12
-20
-18
-6
11
21
20
29
42
45
40
30
21
18
16
9
1
-4
1
8
5
-4
-13
-21
-36
-53
-72
-95
-110
-117
-121
-125
-124
-114
-101
-89
-75
-66
-62
-60
-57
-62
-73
-80
-89
-95
-103
-113
-113
-115
-115
-114
-120
-122
-119
-119
-125
-133
-133
-132
-137
-139
-137
-131
-119
-118
-116
-104
-95
-89
-86
-84
-80
-80
-81
-87
-94
-98
-102
-102
-106
-110
-111
-113
-107
-104
-104
-101
-99
-95
-94
-92
-95
-103
-100
-95

In [ ]:
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path

#give the file path to your audio file
audio_file_path = audio
wav_fpath = audio

wav = preprocess_wav(wav_fpath)
encoder = VoiceEncoder("cpu")
_, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)
print(cont_embeds.shape)

# At this step we have our embeddings in count_embeds, our splits in wav_splits(regading window size)

Loaded the voice encoder model on cpu in 0.01 seconds.
(361, 256)


In [ ]:
from pydub import AudioSegment

def mp3_to_wav(audio_file_path):
    sound = AudioSegment.from_mp3(audio_file_path)
    audio_file_path = audio_file_path.split('.')[0] + '.wav'
    sound.export(audio_file_path, format="wav")
    return audio_file_path

audio_file_path = mp3_to_wav(audio_file_path)
print(audio_file_path)

NameError: ignored

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from spectralcluster import SpectralClusterer

clusterer = SpectralClusterer(
    min_clusters=2,
    max_clusters=100,
    
   
    
    )

labels = clusterer.predict(cont_embeds)
print(labels)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 1 1 1 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
def create_labelling(labels,wav_splits):
    from resemblyzer.audio import sampling_rate
    times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
    labelling = []
    start_time = 0

    for i,time in enumerate(times):
        if i>0 and labels[i]!=labels[i-1]:
            temp = [str(labels[i-1]),start_time,time]
            labelling.append(tuple(temp))
            start_time = time
        if i==len(times)-1:
            temp = [str(labels[i]),start_time,time]
            labelling.append(tuple(temp))
           
    return labelling
  
labelling = create_labelling(labels,wav_splits)
print(labelling)

NameError: ignored

In [ ]:
import matplotlib.pyplot as plt
myList= labelling[1:4]
y= (labelling[0][0],labelling[1][0],labelling[2][0],labelling[3][0],labelling[4][0], labelling[5][0] ,labelling[6][0] , labelling[7][0], labelling[8][0] , labelling[9][0] ,labelling[10][0])
x1=(labelling[0][1],labelling[1][1],labelling[2][1],labelling[3][1],labelling[4][1], labelling[5][1] , labelling[6][1] , labelling[7][1], labelling[8][1] ,  labelling[9][1], labelling[10][1]) 
x2=(labelling[0][2] , labelling[1][2] , labelling[2][2] , labelling[3][2] , labelling[4][2] , labelling[5][2] , labelling[6][2] ,labelling[7][2] , labelling[8][2] ,labelling[9][2], labelling[10][2]) 
print(myList)
print(myList[1][1])
print(y,x1,x2)
# print(x)

plt.plot(x1,x2,y,'ro' ,linewidth=2.0)
#plt.axis([0, 1, 0, 1])
plt.show()

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.1.3-5ubuntu0.6).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 184 kB of archives.
After this operation, 891 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Fetched 184 kB in 1s (229 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 155219 files and directories currently installed.)
Preparing to 

In [ ]:
print(res)

In [ ]:
!git clone 'https://people.csail.mit.edu/hubert/git/pyaudio.git'

Cloning into 'pyaudio'...
remote: Counting objects: 922, done.
remote: Compressing objects: 100% (606/606), done.
remote: Total 922 (delta 571), reused 456 (delta 275)
Receiving objects: 100% (922/922), 343.10 KiB | 2.38 MiB/s, done.
Resolving deltas: 100% (571/571), done.


In [ ]:
pip install sounddevice

In [ ]:
import numpy as np
import sounddevice as sd

fs = 44100
data = np.random.uniform(-1, 1, fs)
sd.play(data, fs)


PortAudioError: ignored

In [ ]:
!pip install PyAudio
!pip install pyaudio

  ERROR: Failed building wheel for PyAudio
  Running setup.py clean for PyAudio
Failed to build PyAudio
    Running setup.py install for PyAudio ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-bse82e8_/pyaudio_be969e63dff043219562cf3a6da87413/setup.py'"'"'; __file__='"'"'/tmp/pip-install-bse82e8_/pyaudio_be969e63dff043219562cf3a6da87413/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-ll32d30g/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/PyAudio Check the logs for full command output.
  Using cached PyAudio-0.2.11.tar.gz (37 kB)
  ERROR: Failed building wheel

In [ ]:
pip install PyAudio

  Using cached PyAudio-0.2.11.tar.gz (37 kB)
  ERROR: Failed building wheel for PyAudio
  Running setup.py clean for PyAudio
Failed to build PyAudio
    Running setup.py install for PyAudio ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-uanoqjc_/pyaudio_37b57e7cdf954279b105cc09149a7e37/setup.py'"'"'; __file__='"'"'/tmp/pip-install-uanoqjc_/pyaudio_37b57e7cdf954279b105cc09149a7e37/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-95pu1typ/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/PyAudio Check the logs for full command output.


In [ ]:

!pip install PyAudio
!pip install pipwin
!pipwin install pyaudio

  Using cached PyAudio-0.2.11.tar.gz (37 kB)
  ERROR: Failed building wheel for PyAudio
  Running setup.py clean for PyAudio
Failed to build PyAudio
    Running setup.py install for PyAudio ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-s_e6361p/pyaudio_69b3c5d558414caab4853bf773adae75/setup.py'"'"'; __file__='"'"'/tmp/pip-install-s_e6361p/pyaudio_69b3c5d558414caab4853bf773adae75/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-mz4xbpc1/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/PyAudio Check the logs for full command output.
     |████████████████████████

In [ ]:
!pip install PyAudio
!pip install pyaudio

  Using cached PyAudio-0.2.11.tar.gz (37 kB)
  ERROR: Failed building wheel for PyAudio
  Running setup.py clean for PyAudio
Failed to build PyAudio
    Running setup.py install for PyAudio ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-yrs30et1/pyaudio_6f4984f02c1c4fd7a502425235d079ad/setup.py'"'"'; __file__='"'"'/tmp/pip-install-yrs30et1/pyaudio_6f4984f02c1c4fd7a502425235d079ad/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-cfothie3/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/PyAudio Check the logs for full command output.
  Using cached PyAudio-0.2.11.

In [ ]:
import pyaudio
import struct
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: ignored